In [1]:
import matplotlib.pyplot as plt

import  torch.utils.data as data_utils
import torch
import torch.nn as nn
from torchvision.models import inception
from torchvision.models import Inception3
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets,models,transforms
import torch.optim as optim


import numpy as np
import random
import gc

import copy
import torch
import numpy as np
from torch.autograd import Variable
import sys
import os
import scipy
import cv2
from PIL import Image

In [116]:
res_tf = models.resnet18(pretrained=True)
alex_tf = models.alexnet(pretrained=True)

In [117]:
res_new = models.resnet18(pretrained=False)
alex_new = models.alexnet(pretrained=False)

In [2]:
res = models.resnet18(pretrained=True)
res.fc = nn.Linear(res.fc.in_features,15)

In [6]:
class model_pip(object):

    def __init__(self,model_in,data_path = '/data/gabriel/OCR/OCR_data/',batch_size=90,lr = 0.001,epochs=10, 
                 gpu=0,f_extractor = False,scale = False,
                 criterion=nn.CrossEntropyLoss(),use_gpu=True,lr_decay_epoch=7):
        self.epochs=int(epochs)
        self.lr_decay_epoch=lr_decay_epoch
        self.data_path = data_path
        self.model = model_in
        self.b_size = batch_size
        self.scale = scale
        self.lr = lr
        self.criterion = criterion
        self.fe = f_extractor
        self.gpu=gpu
        self.use_gpu=use_gpu
    def transform(self):
        if(self.scale==True):
    
            data_transforms = {'train':transforms.Compose([transforms.Scale(300),
                                            transforms.CenterCrop(300),
                                               #transforms.RandomHorizontalFlip(),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
                   ,
                   'val':transforms.Compose([transforms.Scale(300),
            transforms.CenterCrop(300),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
            }
        else:
            data_transforms = {'train':transforms.Compose([transforms.CenterCrop((646,434)),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
                   ,
                   'val':transforms.Compose([
            transforms.CenterCrop((646,434)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
            }
        dsets = {x:datasets.ImageFolder(os.path.join(self.data_path,x),data_transforms[x])
            for x in ['train','val']}

        dset_loaders ={x: torch.utils.data.DataLoader(dsets[x],batch_size=self.b_size,shuffle=True,num_workers=4)
              for x in ['train','val']}

        dset_sizes = {x:len(dsets[x]) for x in ['train','val']}
        
        return dsets,dset_loaders,dset_sizes
    
    #@staticmethod
    def lr_scheduler(self,epoch,optimz):
        lr = self.lr*(0.1**(epoch//self.lr_decay_epoch))
        if epoch%self.lr_decay_epoch ==0:
            print('lr = ',lr)
        for param_group in optimz.param_groups:
            param_group['lr'] = lr
        return optimz
    
    #@staticmethod
    def imshow(inp,title=None):
        inp=inp.numpy().transpose((1,2,0))
        mean=np.array([0.485,0.456,0.406])
        std = np.array([0.229,0.224,0.225])
        inp = std*inp + mean
        plt.imshow(inp),plt.show()
        plt.pause(0.01)
    
    
    
    def train_model(self, n=None):
        dsets,dset_loaders,dset_sizes = self.transform() 
        print(dset_sizes)
        model = self.model
        best_model=model
        if(torch.cuda.is_available() and self.use_gpu):
            torch.cuda.set_device(self.gpu)
            model=model.cuda()
        
        best_acc = 0.0
        best_epoch = 0
        op = optim.SGD(model.parameters(),lr = self.lr,momentum=0.9)
        
        criterion = self.criterion
        if(~self.fe):
        
            for epoch in range(0,self.epochs):
                print('Epoch = ',epoch)

                for phase in ['train','val']:
                    if(phase == 'train'):
                        model.train(True)

                        op = self.lr_scheduler(epoch,op)
                    else:
                        model.train(False)
                    c_mat = np.zeros((15,15)) 
                    running_loss = 0.0
                    running_corrects = 0.0
                    running_tp = 0.0
                    for data in dset_loaders[phase]:
                        inputs,labels = data
                        print(inputs.size())
                        if(torch.cuda.is_available() and self.use_gpu):
                            inputs,labels = Variable(inputs.cuda()),Variable(labels.cuda())
                        else:
                            inputs,labels = Variable(inputs),Variable(labels)
                        
                        op.zero_grad()
                        flag=0
                        
                        if(inputs.size(0)<self.b_size and  n == 'Inception'):
                            flag=1
                            
                            if(torch.cuda.is_available() and self.use_gpu):   
                                temp = Variable(torch.zeros((self.b_size,3,300,300)).cuda())
                                temp2 = Variable(torch.LongTensor(self.b_size).cuda())
                                             
                            else:
                                temp=Variable(torch.zeros((self.b_size,3,300,300)))
                                temp2=Variable(torch.LongTensor(self.b_size))
                                             
                            temp[0:inputs.size(0)]=inputs
                            inputs = temp
                            
                            temp2[0:labels.size(0)] = labels
                            temp2[labels.size(0):] = 0
                            labels = temp2
                        
                        outputs = model(inputs)
                        print(outputs.size())
                        print(labels.size())
                        if(n=='Inception'):
                            if phase=='val':

                                _,preds = torch.max(outputs.data,1)
                                loss = criterion(outputs,labels)
                            else:
                                _,preds = torch.max(outputs[0].data,1)
                                loss = criterion(outputs[0],labels)

                        else:
                            _,preds = torch.max(outputs.data,1)
                            loss = criterion(outputs,labels)



                        if phase=='train':
                            loss.backward()
                            op.step()

                        running_loss+=loss.data[0]
                        running_corrects += torch.sum(preds == labels.data)
                        for i in range(0,labels.data.cpu().numpy().shape[0]):

                            c_mat[labels.data.cpu().numpy()[i],preds.cpu().numpy()[i]]+=1


                    epoch_loss = running_loss/dset_sizes[phase]
                    epoch_acc = running_corrects/dset_sizes[phase]
                    #epoch_tpr = running_tp/dset_sizes[phase]
                    print(phase + '\n')
                    print(phase + '{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
                    #print(c_mat)
                    print(c_mat)
                    if phase=='val' and epoch_acc>best_acc:
                        best_acc=epoch_acc
                        best_model=copy.deepcopy(model)
                        best_epoch=epoch

                #print()

            print(best_acc)
            print(best_epoch)
            return best_model.cpu()

In [9]:
obj = model_pip(res,scale=True,batch_size=90)


In [10]:
model = obj.train_model()
#a,b,c = obj.transform()

{'train': 90, 'val': 240}
Epoch =  0
lr =  0.001
torch.Size([90, 3, 300, 300])
torch.Size([90, 15])
torch.Size([90])
train

traintrain Loss: 0.0106 Acc: 0.9889
[[ 6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  5.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  0.  0.]
 [ 0.  0.  0.  0.  0.

In [124]:
print(c)

{'train': 90, 'val': 240}


In [79]:
for i,j in (b['train']):
    print(i.size())
    print(j.size())
    break

torch.Size([10, 3, 300, 300])
torch.Size([10])
